In [1]:
from mcmc import *
from fit import *
from process_showers import ProcessEvents
from counter_config import CounterConfig, init_config
# from datafiles import *
import matplotlib.pyplot as plt
plt.ion()
from utils import plot_event, plot_generator


data_date_and_time = '20200228054423'
cfg = init_config(data_date_and_time)
# ckv = GetCkv(cfg)

In [2]:
# pars = [600.,2.e6,np.deg2rad(40.),np.deg2rad(315.), 450., -660.,-29,0,70,80.]
# ev = BasicParams.get_event(pars)
# pe = ProcessEvents(cfg, frozen_noise=False)
# real_nfits = pe.gen_nfits_from_event(ev)

In [3]:
corsika_file = '/home/isaac/NIZ/angle_cut_no_thinning/iact_DAT000064.dat'
pe = ProcessEvents(cfg, frozen_noise=True)
real_nfits = pe.gen_nfits_from_ei(corsika_file)

In [4]:
pf = NichePlane(real_nfits)
ty = tyro(real_nfits)

In [5]:
pf.theta

0.6421481780386517

In [6]:
pf.phi

5.411703623988073

In [7]:
import CHASM as ch
ei = ch.EventioWrapper(corsika_file)
xmax = ei.X[ei.nch.argmax()]
nmax = ei.nch.max()
theta = ei.theta
phi = ei.phi
pars = [xmax,nmax,theta,phi,437., -660.,-29.0,0,70,0.]
print(pars)

[525.0, 1200354.0, 0.7038913, 4.981169732409068, 437.0, -660.0, -29.0, 0, 70, 0.0]


In [8]:
guess = make_guess(ty,pf,cfg)
pardict = {p.name:pars[i] for i,p in enumerate(guess[:-1])}
pardict['t_offset'] = 64.

In [9]:
pt = PeakTimes(real_nfits, BasicParams, cfg)
pt.target_parameters = ['zenith','azimuth']
m = init_minuit(pt, guess)
m.tol = .1
# m.simplex()

In [10]:
nmaxguess = update_guess(m)
pa = NormalizedPulseArea(real_nfits, BasicParams, cfg)
pa.target_parameters = ['xmax','nmax','zenith','azimuth','corex','corey']
m = init_minuit(pa, nmaxguess)
m.simplex()

┌─────────────────────────────────────────────────────────────────────────┐
│                               Simplex                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 31.43 (χ²/ndof = 10.5)     │              Nfcn = 158              │
│ EDM = 0.0785 (Goal: 0.1)         │           time = 58.7 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Hesse not run           │            NO covariance             │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │    446    │    10     │            │            │   400   │   800   │       │
│ 1 │ nmax     │  1.40e6   │  0.07e6   │            │            │  10000  │  1e+08  │       │
│ 2 │ zenith   │  0.7360   │  0.0016   │            │            │    0    │0.742148 │       │
│ 3 │ azimuth  │   5.470   │   0.006   │            │            │ 5.3117  │ 5.5117  │       │
│ 4 │ corex    │   383.6   │    0.9    │            │            │  379.5  │  577.4  │       │
│ 5 │ corey    │  -650.9   │    0.9    │            │            │  -821   │ -619.1  │       │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │     0     │    10     │            │            │  -450   │   450   │  yes  │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

Now with a good estimate for the angles, we can fit the pulse widths to estimate xmax.

In [11]:
# guess = update_guess_values(guess, m)
# pw = PulseWidth(real_nfits, BasicParams, cfg)
# pw.target_parameters = ['xmax']
# m = init_minuit(pw, guess)
# m.simplex(ncall=40)

Now, with a good estimate for log(xmax), we can fit the integrated pulse areas for nmax.

In [12]:
# guess = update_guess_values(guess, m)
# pa = PulseArea(real_nfits, BasicParams, cfg)
# pa.target_parameters = ['nmax']
# m = init_minuit(pa, guess)
# m.simplex(ncall=20)

Now, with these values in the ballpark, we can simultaneously minimize both the shower profile and core position by fitting the normalized pulse areas.

In [13]:
# guess = update_guess_values(guess, m)
# pa = NormalizedPulseArea(real_nfits, BasicParams, cfg)
# pa.target_parameters = ['xmax','nmax','corex','corey']
# m = init_minuit(pa, guess)
# m.simplex()

In [14]:
# guess = update_guess(m)
# at = AllTunka(real_nfits, BasicParams, cfg)
# at.target_parameters = ['t_offset']
# m = init_minuit(at, guess)
# m.migrad()

In [15]:
# guess = update_guess(m)
# at = AllSamples(real_nfits, BasicParams, cfg)
# at.target_parameters = ['t_offset']
# m = init_minuit(at, guess)


In [16]:
# m.tol=.1
# m.fixed = True
# m.fixed['xmax'] = False
# m.fixed['nmax'] = False
# m.fixed['zenith'] = False
# m.fixed['azimuth'] = False
# # m.fixed['corex'] = False
# # m.fixed['corey'] = False
# m.fixed['t_offset'] = False
# # m.scan()
# m.simplex()

In [17]:
guess = update_guess_values(guess, m)
guessdict = {p.name:p for p in guess}
# guessdict['xmax'].limits = (guessdict['xmax'].value - 50., guessdict['xmax'].value + 50.)
# guessdict['nmax'].limits = (guessdict['nmax'].value - 2.e5, guessdict['nmax'].value + 2.e5)
# guessdict['zenith'].limits = (guessdict['zenith'].value - 5.e-3, guessdict['zenith'].value + 5.e-3)
# guessdict['azimuth'].limits = (guessdict['azimuth'].value - 5.e-3, guessdict['azimuth'].value + 5.e-3)
# guessdict['corex'].limits = (guessdict['corex'].value - 5., guessdict['corex'].value + 5.)
# guessdict['corey'].limits = (guessdict['corey'].value - 5., guessdict['corey'].value + 5.)
# guessdict['zenith'].fixed = True
# guessdict['azimuth'].fixed = True
# guessdict['corex'].fixed = True
# guessdict['corey'].fixed = True
guessdict['corez'].fixed = True
guessdict['x0'].fixed = True
guessdict['lambda'].fixed = True
# guessdict['t_offset'].fixed = True

In [18]:
initial_guessdict = {p.name:p.value for p in guess}
initial_guessdict['t_offset'] = 80.

In [19]:
names = [p.name for p in guess if not p.fixed]
ndim = len(names)

In [20]:
sampler, pos, prob, state = main(pa,guess,niter=5000,nwalkers=ndim*2)

  0%|                                       | 13/5000 [01:06<6:49:39,  4.93s/it]/home/isaac/NIZ/niche_fit.py:101: RuntimeWarning: invalid value encountered in power
  lout[fl5] += pk*np.exp(-f5**(2+f5/2))
  1%|▏                                      | 29/5000 [02:10<5:11:58,  3.77s/it]/home/isaac/NIZ/niche_fit.py:101: RuntimeWarning: invalid value encountered in power
  lout[fl5] += pk*np.exp(-f5**(2+f5/2))
  1%|▎                                      | 38/5000 [02:47<5:23:28,  3.91s/it]/home/isaac/NIZ/niche_fit.py:101: RuntimeWarning: invalid value encountered in power
  lout[fl5] += pk*np.exp(-f5**(2+f5/2))
  1%|▍                                      | 53/5000 [03:48<5:10:03,  3.76s/it]/home/isaac/NIZ/niche_fit.py:101: RuntimeWarning: invalid value encountered in power
  lout[fl5] += pk*np.exp(-f5**(2+f5/2))
  1%|▍                                      | 64/5000 [04:34<5:49:25,  4.25s/it]/home/isaac/NIZ/niche_fit.py:101: RuntimeWarning: invalid value encountered in power
  lout[fl5] += 

KeyboardInterrupt: 

In [ ]:
flat_samples = sampler.get_chain(discard=4000,flat=True)

samples = sampler.flatchain
best_sample = samples[np.argmax(sampler.flatlnprobability)]

fig = corner.corner(flat_samples,labels=names,quantiles=[0.16, 0.5, 0.84])

axes = np.array(fig.axes).reshape((ndim, ndim))

# Loop over the diagonal
for i,name in enumerate(names):
    ax = axes[i, i]
    ax.axvline(pardict[name], color="g", label='thrown')
    ax.axvline(initial_guessdict[name], color="b", label='initial guess')
    ax.axvline(best_sample[i], color="r", label = 'max likelihood')
    if i == 0:
        fig.legend()
    # ax.set_xlim(np.min([pardict[names[i]],initial_guessdict[names[i]],best_sample[i]]),np.max([pardict[names[i]],initial_guessdict[names[i]],best_sample[i]]))

    # Loop over the histograms
for yi in range(ndim):
    for xi in range(yi):
        ax = axes[yi, xi]
        # ax.set_xlim(np.min([pardict[names[xi]],initial_guessdict[names[xi]],best_sample[xi]]),np.max([pardict[names[xi]],initial_guessdict[names[xi]],best_sample[xi]]))
        # ax.set_ylim(np.min([pardict[names[yi]],initial_guessdict[names[yi]],best_sample[yi]]),np.max([pardict[names[yi]],initial_guessdict[names[yi]],best_sample[yi]]))
        ax.axvline(pardict[names[xi]], color="g")
        ax.axvline(best_sample[xi], color="r")
        ax.axhline(pardict[names[yi]], color="g")
        ax.axhline(best_sample[yi], color="r")
        ax.axvline(initial_guessdict[names[xi]], color="b")
        ax.axhline(initial_guessdict[names[yi]], color="b")
        ax.plot(pardict[names[xi]], pardict[names[yi]], "sg")
        ax.plot(best_sample[xi], best_sample[yi], "sr")
        ax.plot(initial_guessdict[names[xi]], initial_guessdict[names[yi]], "sb")

In [ ]:
pars

In [ ]:
guessdict = {p.name:p.value for p in guess}

In [ ]:
np.array(list(guessdict.values()))

In [ ]:
len(flat_samples)

In [ ]:
best_sample

In [ ]:
len(real_nfits)